In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='3.128.232.8:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

### Constitutive gene

In [ ]:
dna = fj.get('dna', name='Const')
if len(dna)==0:
    dna = fj.create('dna', name='Const')
vector = fj.get('vector', name='Const')    
if len(vector)==0:
    vector = fj.create('vector', name='Const', dnas=dna.id)
    
const = GeneticNetwork(vector=vector.id[0])

In [ ]:
cfp = fj.get('signal', name='CFP')
sfp1 = Reporter(name='CFP', degradation_rate=0, signal_id=cfp.id[0])

const.add_reporter(sfp1)

In [ ]:
const.add_operator(Source(output=sfp1, rate=1))

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 4)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 4)

def profile(biomass, gr, t):
    return gr * 10 + 1
    
metab = SimulatedMetabolism(biomass, growth_rate, profile)

In [ ]:
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

sample = Sample(circuit=const, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay([sample], 
              n_measurements=100, 
              interval=0.25,
              name='Loica constitutive expression',
              description='Simulated constitutive gene generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Testing')
assay.upload(fj, study.id[0])

In [ ]:
source = const.operators[0]
source.characterize(
    fj,
    vector=vector.id,
    media=media.id,
    strain=strain.id,
    signal=cfp.id,
    biomass_signal=biomass_signal.id,
    fmax=0.1,
    epsilon=0
)

In [ ]:
print('Protein degradation rate = ', source.gamma)
plt.plot(source.profile)

### Receiver

Create a genetic network and associate it with a Flapjack vector:

In [ ]:
dna = fj.get('dna', name='Rec')
if len(dna)==0:
    dna = fj.create('dna', name='Rec')
vector = fj.get('vector', name='Rec')    
if len(vector)==0:
    vector = fj.create('vector', name='Rec', dnas=dna.id)
    
rec = GeneticNetwork(vector=vector.id[0])

Create a reporter and associate it with a Flapjack signal so we can record the behaviour of the circuit:

In [ ]:
cfp = fj.get('signal', name='CFP')
sfp1 = Reporter(name='CFP', degradation_rate=0, signal_id=cfp.id[0])

rec.add_reporter(sfp1)

Create a regulator to receive the inducer:

In [ ]:
receptor = Regulator(name='Receptor', degradation_rate=1)
rec.add_regulator(receptor)
rec.add_operator(Source(output=receptor, rate=1))

Create and add a receiver operator to the circuit, linking it to an AHL supplement and the receptor we just created:

In [ ]:
ahl = Supplement(name='AHL')
rec.add_operator(Receiver(receptor=receptor, inducer=ahl, output=sfp1, a=0, b=100, K=1, n=2))

Now we have constructed the circuit we need to run an assay containing some samples. The sample is driven by a metabolism which defines the dynamics of growth and gene expression profiles:

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 4)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 4)

def profile(biomass, gr, t):
    return gr * 10 + 1
    
metab = SimulatedMetabolism(biomass, growth_rate, profile)

Next we create a set of samples associated to Flapjack media and strain, and containing our AHL at different concentrations

In [ ]:
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

# Create list of samples    
samples = []
for conc in np.logspace(-3, 3, 12):
    sample = Sample(circuit=rec, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)

Now we can create and run the assay:

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.25,
              name='Loica receiver',
              description='Simulated receiver generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

Plot the results:

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

Upload the simulated data to flapjack

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')

assay.upload(fj, study.id[0])

In [ ]:
study = fj.get('study', name='Loica testing')
assay = fj.get('assay', study=study.id[0], temperature=0)
meas = fj.measurements(
    media=media.id,
    strain=strain.id,
    vector=vector.id
)
meas

### Inverter

In [ ]:
dna = fj.get('dna', name='Inv')
if len(dna)==0:
    dna = fj.create('dna', name='Inv')
vector = fj.get('vector', name='Inv')    
if len(vector)==0:
    vector = fj.create('vector', name='Inv', dnas=dna.id)
    
inv = GeneticNetwork(vector=vector.id[0])

In [ ]:
inv.add_regulator(receptor)
inv.add_operator(Source(output=receptor, rate=1))

In [ ]:
inv.add_reporter(sfp1)
laci = Regulator(name='LacI', degradation_rate=1)
inv.add_regulator(laci)
inv.add_operator(Not(input=laci, output=sfp1, a=100, b=0, K=1, n=2))

In [ ]:
inv.add_operator(
    Receiver(receptor=receptor, inducer=ahl, output=laci, a=0, b=100, K=1, n=2)
)

In [ ]:
# Create list of samples    
samples = []
for conc in np.logspace(-3, 3, 12):
    sample = Sample(circuit=inv, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)

In [ ]:
biomass_signal = fj.get('signal', name='OD')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.25,
              name='Loica inverter',
              description='Simulated inverter generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')

assay.upload(fj, study.id[0])

### Repressilator

In [ ]:
rep = GeneticNetwork()

laci = Regulator(name='LacI', degradation_rate=1, init_concentration=5)
tetr = Regulator(name='TetR', degradation_rate=1)
ci = Regulator(name='cI', degradation_rate=1)
rep.add_regulator(laci)
rep.add_regulator(tetr)
rep.add_regulator(ci)

sfp1 = Reporter(name='SFP1', degradation_rate=1)
rep.add_reporter(sfp1)
sfp2 = Reporter(name='SFP2', degradation_rate=1)
rep.add_reporter(sfp2)
sfp3 = Reporter(name='SFP3', degradation_rate=1)
rep.add_reporter(sfp3)

rep.add_operator(Not(input=ci, output=laci, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=laci, output=tetr, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=tetr, output=ci, a=100, b=0, K=1, n=2))

rep.add_operator(Not(input=ci, output=sfp1, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=laci, output=sfp2, a=100, b=0, K=1, n=2))
rep.add_operator(Not(input=tetr, output=sfp3, a=100, b=0, K=1, n=2))

In [ ]:
sample = Sample(circuit=rep, metabolism=metab)
assay = Assay([sample], n_measurements=100, interval=0.25)
assay.run()

In [ ]:
assay.measurements

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='SFP1'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='SFP2'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='SFP3'].plot(x='Time', y='Measurement', ax=ax)

### Toggle switch

In [ ]:
dna = fj.create('dna', name='Toggle')
vector = fj.create('vector', name='Toggle', dnas=dna.id)
tog = GeneticNetwork(vector=vector.id[0])

laci = Regulator(name='LacI', degradation_rate=1, init_concentration=0.1)
ci = Regulator(name='cI', degradation_rate=1)
tog.add_regulator(laci)
tog.add_regulator(ci)

cfp = fj.get('signal', name='CFP')
yfp = fj.get('signal', name='YFP')
sfp1 = Reporter(name='CFP', degradation_rate=1, signal_id=cfp.id[0])
tog.add_reporter(sfp1)
sfp2 = Reporter(name='YFP', degradation_rate=1, signal_id=yfp.id[0])
tog.add_reporter(sfp2)

tog.add_operator(Not(input=laci, output=ci, a=10, b=0, K=1, n=2))
tog.add_operator(Not(input=ci, output=laci, a=10, b=0, K=1, n=2))

tog.add_operator(Not(input=ci, output=sfp1, a=10, b=0, K=1, n=2))
tog.add_operator(Not(input=laci, output=sfp2, a=10, b=0, K=1, n=2))

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

biomass_signal = fj.get('signal', name='OD')
sample = Sample(circuit=tog, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0]
               )
assay = Assay([sample], 
              n_measurements=100, 
              interval=0.25,
              name='Loica toggle',
              description='Simulated toggle switch generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='YFP'].plot(x='Time', y='Measurement', ax=ax)

### Upload simulated data to flapjack

In [ ]:
assay.upload(fj, study.id[0])